tar -C /tmp/ayu -zvxf xyz.tar.gz

In [1]:
import os,sys,pickle,time
import pandas as pd
import numpy as np
import subprocess
from multiprocessing import Pool 
import shutil
from tqdm import tqdm

In [3]:
tgz_dir = "../../RAID5/pcap/"
decompress_dir = "../../RAID6/pcap_process/"
all_dates = next(os.walk(tgz_dir))[1]
all_dates = sorted(all_dates)
all_dates

['2019_12_29',
 '2019_12_30',
 '2019_12_31',
 '2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_05',
 '2020_01_06',
 '2020_01_07',
 '2020_01_08',
 '2020_01_09',
 '2020_01_10',
 '2020_01_11',
 '2020_01_12',
 '2020_01_13',
 '2020_01_14',
 '2020_01_15',
 '2020_01_16',
 '2020_01_17',
 '2020_01_18',
 '2020_01_19',
 '2020_01_20',
 '2020_01_21',
 '2020_01_22',
 '2020_01_23',
 '2020_01_24',
 '2020_01_25',
 '2020_01_26',
 '2020_01_27',
 '2020_01_28',
 '2020_01_29',
 '2020_01_30',
 '2020_01_31',
 '2020_02_01',
 '2020_02_02',
 '2020_02_03',
 '2020_02_04',
 '2020_02_05',
 '2020_02_06',
 '2020_02_07',
 '2020_02_08',
 '2020_02_09',
 '2020_02_10',
 '2020_02_11',
 '2020_02_12',
 '2020_02_13',
 '2020_02_14',
 '2020_02_15',
 '2020_02_16',
 '2020_02_17',
 '2020_02_18',
 '2020_02_19',
 '2020_02_20',
 '2020_02_21',
 '2020_02_22',
 '2020_02_23',
 '2020_02_24',
 '2020_02_25',
 '2020_02_26',
 '2020_02_27',
 '2020_02_28',
 '2020_02_29',
 '2020_03_01',
 '2020_03_02',
 '2020_03_03',
 '2020_03_

In [4]:
def parallel_untar(in_file,out_dir):
    test_dir = out_dir.split('/')
    test_dir = test_dir[:3]
    test_dir = "/".join(test_dir)
    total, used, free = shutil.disk_usage(test_dir)
    if (free<total*0.01) or (free // (2**30) < 130):
#         cmd = ['rmdir',out_dir]
#         p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
#         out, err = p.communicate()
#         print("!Error:", err, '!')
        print('Disk not enough space ERR:',in_file,out_dir)
        return out_dir
    else:
        if not os.path.exists(out_dir):
            os.makedirs(out_dir, exist_ok=True)
        
        
        #cat pcap_2020_04_18_part* > pcap_2020_04_18.tar.gz # 合併
        #cat pcap_2020_04_18_part* | tar -C out_dir -zxvf #解壓
        
        cmd = ["tar","-C", out_dir,
           "-zvxf", in_file]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
        out, err = p.communicate()
        print("!Error:", err, '! Done:',out_dir)
    

## For ISP's Pcap
parallel by function name

In [10]:
# %%time
in_file_paths = []
out_dir_paths = []
days = ['2020_01_20','2020_01_21'] #指定日期，清空掉就是全部日子
isps = ['中華電信','中嘉寬頻','亞太電信','凱擘','台固媒體','台灣之星','台灣固網','台灣基礎開發','台灣大哥大',
       '台灣碩網','遠傳電信']
# isps = ['台灣固網','台固媒體']
for date in tqdm(all_dates):
    if date not in days:
        continue
    pcap_path = tgz_dir + date + '/'
    isp_names = next(os.walk(pcap_path))[1]
    take_isps = []
    for isp in isp_names:
        if isp in isps:
            take_isps.append(isp)
    for isp in take_isps:
        file_names = next(os.walk(pcap_path+isp))[2]
        file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
        if len(file_names)>1:
            print('WARNING: more than 1 pcap tgz file in',pcap_path+isp+'/') #可能分part
        for file_name in file_names:
            pcap_path_ = pcap_path+isp+'/'+file_name
            output_path = decompress_dir+date+'/'+isp+'/'+file_name.replace('.tar.gz','_dir')+'/'
            in_file_paths.append(pcap_path_)
            out_dir_paths.append(output_path)

assert len(in_file_paths) == len(out_dir_paths)
assert in_file_paths[0].split('/')[-3] == out_dir_paths[0].split('/')[-4]
assert in_file_paths[-1].split('/')[-2] == out_dir_paths[-1].split('/')[-3]
len(in_file_paths)

100%|██████████| 70/70 [00:00<00:00, 32299.37it/s]


22

In [11]:
%%time
load_start = time.time()
pool_result = []
pool = Pool(processes=3) #processes=21
for path_in,dir_out in zip(in_file_paths,out_dir_paths):
    r = pool.apply_async(parallel_untar,args=(path_in,dir_out))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r.get())
load_end = time.time() - load_start
print("Decompress Time:", '{:02f}:{:02f}:{:02f}'.format(load_end // 3600, (load_end % 3600 // 60), load_end % 60))

!Error: b'tar: ./snort.log.1579552561: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579567726: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579607014: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579539547: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579581178: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579612197: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579581025: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579615268: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579586681: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579621172: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579608374: Cannot chan

 ! Done: ../../RAID6/pcap_process/2020_01_21/中嘉寬頻/pcap_2020_01_21_dir/


!Error: b'tar: ./snort.log.1579583902: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579562317: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579583450: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579546115: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579571932: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579569967: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579573169: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579573534: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579565305: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579569896: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579587306: Cannot chan

!Error: b'tar: ./snort.log.1579556913: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579589558: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579609329: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579553143: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579558428: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579571494: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579554779: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579579407: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579537607: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579540479: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579607912: Cannot chan

! Done: ../../RAID6/pcap_process/2020_01_21/台灣大哥大/pcap_2020_01_21_dir/


!Error: b'tar: ./snort.log.1579613168: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579564893: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579614078: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579594129: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579563855: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579597414: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579584559: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579615159: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579575544: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579559757: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1579587481: Cannot chan

## For ISPs' honeypot

In [22]:
paths = []
days = ['2020-01-09','2020-01-10','2020-01-11'] #指定日期，清空掉就是全部日子
for name in tqdm(company_names):
    if name != '中華電信': #指定電信公司，註解掉就是全部
        continue
#     print('=== Running:',name,'===')
    hp_path = root_dir + name + '/honeypot/'
    date_dir_names = next(os.walk(hp_path))[1]
    if days != []:
        temp = []
        for date in date_dir_names:
            for day in days:
                if day == date:
                    temp.append(date)
        date_dir_names = temp
    else:
        pass
    for date in date_dir_names:
        hp_date_path = hp_path + date +'/'
        hp_names_dir = next(os.walk(hp_date_path))[1]
        for name in hp_names_dir:
            file_names = next(os.walk(hp_date_path+name+'/'))[2]
            file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
            new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
            for dir_name in new_dir_names:
                path = hp_date_path+name+'/'+dir_name
                os.makedirs(path, exist_ok=True)
                paths.append(path)
        
paths

100%|██████████| 11/11 [00:00<00:00, 420.87it/s]


['../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/glastopf/glastopf_file_2020-01-11_dir',
 '../

In [23]:
%%time
pool_result = []
pool = Pool() #processes=21
for path in paths:
    r = pool.apply_async(parallel_untar,(path,))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r)

!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4390>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4c90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4290>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4490>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4690>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4850>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4310>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163aa90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163ab90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4610>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163a750>
return: <multiprocessing.poo